# Introduction

Traditional job sites can be overwhelming. Additionally, Lambda School graduates can save time and anxiety by focusing on companies who understand the unique value Lambda graduates bring as employees. Therefore we are developing a website where Lambda School students and alumni can post company and interview experiences and find helpful posts that others have made. 

For its first user feature, the Data Science team is developing automated content moderation. As the website scales, manual content moderation may not be a pragmatic way of enforcing content rules. Furthermore, inappropriate content undermines the site's core mission of saving Lambda students time and helping ease their anxieties. Therefore we seek to find a model that can automatically and accurately classify posts that should be flagged or removed.

It is worth noting that many tweets in the data used within this notebook contain hateful or obscene content. Those who may be traumatized by such content may want to avoid reading this notebook.

## Modeling Plan

At the risk of tautology, data science requires data. So we first assess the data we have available and what we can get. We face a classic problem right now in building out features for a fledgling site: because we don't have users, we don't have actual user data; however, we'll never generate enough users to have sufficient data if they're encountering abusive and hateful content. For this reason, we concluded starting with a model trained on external data was a possible route.

We were able to find several labeled data sets that flagged hateful, abusive, or spam content (or some combination of the three). Our initial strategy will be to use these to train, validate, and test models. It is worth noting that all three data sets were composed of Tweets. Our hope is that patterns of abusive text are similar enough across websites that the model learning on this will transfer.

We will load and explore the data, then fit sequentially more complex models in order to maximize predictive performance.

# Load Data

In [1]:
import pandas as pd
import numpy as np
import spacy
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import wandb
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from wandb.keras import WandbCallback
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
import os

In [2]:
nlp = spacy.load("en_core_web_lg")

In [5]:
hundred_k_tweets = pd.read_csv("data\\hatespeech_text_label_vote.csv", sep='\t', header=None, names=["tweet", "category", "votes"])

In [6]:
hundred_k_tweets["inappropriate"] = (hundred_k_tweets["category"].isin(["spam", "abusive", "hateful"]))
hundred_k_tweets = hundred_k_tweets.drop(["category", "votes"], axis=1)

In [7]:
additional_tweets = pd.read_csv("data\\labeled_data.csv")

In [8]:
additional_tweets = additional_tweets.drop(["Unnamed: 0"], axis=1)
additional_tweets["inappropriate"] = additional_tweets["class"] != 2
additional_tweets = additional_tweets.drop(["count", "hate_speech", "offensive_language", "neither", "class"], axis=1)

In [9]:
kaggle_tweets = pd.read_csv("data/train_E6oV3lV.csv")

In [10]:
kaggle_tweets["inappropriate"] = kaggle_tweets["label"]
kaggle_tweets = kaggle_tweets.drop(["id", "label"], axis=1)

In [11]:
dfs = [hundred_k_tweets, additional_tweets, kaggle_tweets]
df = pd.concat(dfs, ignore_index=True)

In [12]:
df = df.drop_duplicates(subset=['tweet', 'inappropriate'])
appropriate = ~df['inappropriate']
dupe_tweet = df.duplicated(subset=['tweet'], keep=False)
df = df[~((dupe_tweet) & (appropriate))].copy()

In [13]:
df.shape

(146264, 2)

In [14]:
df.duplicated(subset=['tweet']).any()

False

In [ ]:
# df.to_csv('combined_deduped.csv', index=False)